# Processing Multisentence Documents

This notebook uses [TextBlob](https://pypi.org/project/textblob/) to do sentence splitting. If you do not have TextBlob installed, you can install it by uncomment the line in the cell below and executing that cell. 

In [ ]:
#!pip install textblob

In [ ]:
import pyConTextNLP.pyConText as pyConText
import pyConTextNLP.itemData as itemData
from textblob import TextBlob
import networkx as nx
import pyConTextNLP.display.html as html
from IPython.display import display, HTML

In [ ]:
reports = [
    """IMPRESSION: Evaluation limited by lack of IV contrast; however, no evidence of
      bowel obstruction or mass identified within the abdomen or pelvis. Non-specific interstitial opacities and bronchiectasis seen at the right
     base, suggestive of post-inflammatory changes.""",
    """IMPRESSION: Evidence of early pulmonary vascular congestion and interstitial edema. Probable scarring at the medial aspect of the right lung base, with no
     definite consolidation."""
    ,
    """IMPRESSION:
     
     1.  2.0 cm cyst of the right renal lower pole.  Otherwise, normal appearance
     of the right kidney with patent vasculature and no sonographic evidence of
     renal artery stenosis.
     2.  Surgically absent left kidney.""",
    """IMPRESSION:  No pneumothorax.""",
    """IMPRESSION: No definite pneumothorax"""
    """IMPRESSION:  New opacity at the left lower lobe consistent with pneumonia."""
]

In [ ]:
modifiers = itemData.get_items(
    "https://raw.githubusercontent.com/chapmanbe/pyConTextNLP/master/KB/lexical_kb_05042016.yml")
targets = itemData.get_items(
    "https://raw.githubusercontent.com/chapmanbe/pyConTextNLP/master/KB/utah_crit.yml")


## Define ``markup_sentence``

We are putting the functionality we went through in the previous two notebooks ([BasicSentenceMarkup](./BasicSentenceMarkup.ipynb) and [BasicSentenceMarkupPart2](BasicSentenceMarkupPart2.ipynb)) into a function ``markup_sentence``. We add one step to the function: ``dropInactiveModifiers`` will delete any modifier node that does not get attached to a target node.

In [ ]:
def markup_sentence(s, modifiers, targets, prune_inactive=True):
    """
    """
    markup = pyConText.ConTextMarkup()
    markup.setRawText(s)
    markup.cleanText()
    markup.markItems(modifiers, mode="modifier")
    markup.markItems(targets, mode="target")
    markup.pruneMarks()
    markup.dropMarks('Exclusion')
    # apply modifiers to any targets within the modifiers scope
    markup.applyModifiers()
    markup.pruneSelfModifyingRelationships()
    if prune_inactive:
        markup.dropInactiveModifiers()
    return markup

In [ ]:
report = reports[0]
print(report)

#### Create a ``ConTextDocument``

``ConTextDocument`` is a class for organizing the markup of multiple sentences. It has a private attribute that is NetworkX DiGraph that represents the document structure. In this exmaple we only use the ``ConTextDocument`` class to collect multiple sentence markups.

In [ ]:
context = pyConText.ConTextDocument()

####  Split the document into sentences and process each sentence

pyConTextNLP comes with a simple sentence splitter in ``helper.py``. I have not been maintaining this and have recently been using TextBlob to split sentences. A known problem with either sentence splitting solution is enumerated lists that don't use periods.

In [ ]:
blob = TextBlob(report.lower())
count = 0
rslts = []
for s in blob.sentences:
    m = markup_sentence(s.raw, modifiers=modifiers, targets=targets)
    rslts.append(m)

for r in rslts:
    context.addMarkup(r)

### Displaying pyConTextNLP Markups

The ``display`` subpackage contains some functionality for visualizing the markups. Here I use HTML to color-code identified concepts.

In [ ]:
clrs = {\
    "bowel_obstruction": "blue",
    "inflammation": "blue",
    "definite_negated_existence": "red",
    "probable_negated_existence": "indianred",
    "ambivalent_existence": "orange",
    "probable_existence": "forestgreen",
    "definite_existence": "green",
    "historical": "goldenrod",
    "indication": "pink",
    "acute": "golden"
}

In [ ]:
display(HTML(html.mark_document_with_html(context,colors = clrs, default_color="black")))

### There is also a rich XML description of the ``ConTextDocument``

In [ ]:
print(context.getXML())